In [1]:
# parameters

time_series = "mackeyglass"  # "mackeyglass", "lorenz", "henonmap"
num_exp = 20
forecast = 10
num_bits = 64
quant_mode = "all" # "res", "all"

# Quantized ESNs

## Librairies

In [2]:
import os
import pickle

import numpy as np

import reservoirpy as rpy
from reservoirpy.mat_gen import random_sparse
from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import to_forecasting
from reservoirpy.datasets import mackey_glass, lorenz, henon_map
from reservoirpy.observables import mse, rmse, nrmse, rsquare

import matplotlib
import matplotlib.pyplot as plt

## Parameters

In [3]:
# randomness
seed = 42
seed_rpy = rpy.set_seed(seed)

# data
warmup = 200
train_len, test_len = 2000, 500
train_len += warmup
timesteps = train_len + test_len + forecast

# model
plot = False

units = 100
spectral_radius = 0.99
leak_rate = 0.3
input_scaling = 1.0
connectivity = 0.1
input_connectivity = 0.2
regularization = 1e-8

## Data

In [4]:
if time_series == "mackeyglass":
    
    tau = 17
    X = mackey_glass(timesteps, tau=tau)
    X = 2 * (X - X.min()) / (X.max() - X.min()) - 1 # rescale ts between -1 and 1
    
elif time_series == "lorenz":
    
    X = lorenz(timesteps)

elif time_series == "henonmap":
    
    X = henon_map(timesteps)

else:
    raise Exception("Time series not defined.")

output_dim = X.shape[1]

In [5]:
def plot_time_series(X, timesteps, forecast):

    fig = plt.figure(figsize=(13, 3))
    N = timesteps

    ax = plt.subplot((121))
    t = np.linspace(0, N, N)
    for i in range(N-1):
        ax.plot(t[i:i+2], X[i:i+2], color=plt.cm.rainbow(255*i//N), lw=1.0)

    plt.title(f"Timeseries - {N} timesteps")
    plt.xlabel("$t$")
    plt.ylabel("$P(t)$")

    ax2 = plt.subplot((122))
    ax2.margins(0.05)
    for i in range(N-1):
        ax2.plot(X[i:i+2], X[i+forecast:i+forecast+2], color=plt.cm.rainbow(255*i//N), lw=1.0)

    plt.title("Phase diagram")
    plt.xlabel(f"$P(t-{forecast})$")
    plt.ylabel("$P(t)$")

    plt.tight_layout()
    plt.show()

In [6]:
if plot:
    plot_time_series(X, timesteps-forecast, forecast)

In [7]:
x, y = to_forecasting(X, forecast=forecast)

X_train, y_train = x[:train_len], y[:train_len]
X_test, y_test = x[train_len:], y[train_len:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2200, 1), (2200, 1), (500, 1), (500, 1))

## Model

In [8]:
def quantize(arr, num_bits, verbose=False):
    """
    Quantize a numpy array to a specified number of bits.
    
    Parameters:
    arr (numpy.ndarray): Input array to be quantized
    num_bits (int): Number of bits for quantization
    
    Returns:
    numpy.ndarray: Quantized array
    """
    # Determine the range of the input array
    min_val, max_val = arr.min(), arr.max()
    
    # Calculate the step size
    steps = 2**num_bits - 1
    step_size = (max_val - min_val) / steps
    
    # Quantize the array
    quantized = np.round((arr - min_val) / step_size)
    
    # Scale back to the original range
    quantized = quantized * step_size + min_val

    if verbose:
        print("Quantization done.")
    return quantized

In [9]:
def W_custom(size, sr, sparsity, num_bits=64):
    
    W = np.random.normal(0, 1, (size, size))
    W[np.random.rand(*W.shape) > sparsity] = 0  # Apply sparsity
    
    eigvals = np.linalg.eigvals(W)
    W *= sr / max(abs(eigvals))    # Normalize spectral radius
    eigvals = np.linalg.eigvals(W)
    print("\nrho =", max(abs(eigvals)))

    W = quantize(W, num_bits=num_bits, verbose=True) # quantization
    
    return W    

def reset_esn(output_dim=1):

    W_res = W_custom(size=units, 
                     sr=spectral_radius, 
                     sparsity=connectivity, 
                     num_bits=num_bits)
    
    reservoir = Reservoir(units=units,
                          input_scaling=input_scaling, 
                          sr=spectral_radius,
                          lr=leak_rate, 
                          rc_connectivity=connectivity,
                          input_connectivity=input_connectivity,
                          W=W_res, # custom weights
                          seed=seed_rpy)
        
    readout   = Ridge(output_dim, ridge=regularization)

    return reservoir >> readout

In [10]:
def get_scores(y_test, y_pred):

    mse_ = mse(y_test, y_pred).item()
    nrmse_ = nrmse(y_test, y_pred).item()
    rmse_ = rmse(y_test, y_pred).item()
    rsquare_ = rsquare(y_test, y_pred).item()
    
    return mse_, nrmse_, rmse_, rsquare_

In [11]:
def experiment(X_train, y_train, X_test, y_test, warmup, num_bits):
    
    # Train
    esn = reset_esn(output_dim=output_dim)
    esn = esn.fit(X_train, y_train, warmup=warmup)
    
    # Quantize Wout
    if quant_mode == "all":
        
        ridge_name = esn.nodes[1].name
        Wout, bias = esn.params[ridge_name].values()
        tmp = quantize(np.vstack([Wout, bias]), num_bits=num_bits)
        Wout = tmp[:-1]
        bias = tmp[-1].reshape(-1, output_dim)
        esn.params[ridge_name]["Wout"] = Wout
        esn.params[ridge_name]["bias"] = bias
        print("Quantization done.")
    
    # Test
    y_pred = esn.run(X_test)
    score = get_scores(y_test, y_pred)
    # print(f"Score: {score}")
    
    return score

In [12]:
np.random.seed(seed)

scores = []

for i in range(num_exp):
    
    score = experiment(X_train, y_train, X_test, y_test, warmup, num_bits)
    scores.append(score)

score_d = {}

for j, k in enumerate(["mse", "nrmse", "rmse", "rsquare"]):
    
    tmp = [scores[i][j] for i in range(len(scores))]
    mean = np.mean(tmp).item()
    std = np.std(tmp).item()

    score_d[k] = [mean, std]    

print(f"Scores: {score_d}")


rho = 0.9900000000000039
Quantization done.


Running Model-0:   0%|                                                                              | 0/1 [00:00<?, ?it/s]
Running Model-0: 2200it [00:00, 23805.70it/s]                                                                             
Running Model-0: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.42it/s]


Fitting node Ridge-0...
Quantization done.


Running Model-0: 500it [00:00, 26278.12it/s]                                                                              



rho = 0.9899999999999956
Quantization done.


...


rho = 0.9900000000000013
Quantization done.


Running Model-9:   0%|                                                                              | 0/1 [00:00<?, ?it/s]
Running Model-9: 2200it [00:00, 25199.00it/s]                                                                             
Running Model-9: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 10.96it/s]


Fitting node Ridge-9...
Quantization done.


Running Model-9: 500it [00:00, 26341.83it/s]                                                                              

Scores: {'mse': [1.3539067110705146e-06, 2.8886798108408104e-07], 'nrmse': [0.0006042555853635369, 6.396948555827167e-05], 'rmse': [0.0011571089524984, 0.00012249727800473864], 'rsquare': [0.9999941712799518, 1.2436090159354816e-06]}


## Results

In [13]:
folder = f"runs/{time_series}_{quant_mode}"

if not os.path.exists(folder):
    os.makedirs(folder)

file = os.path.join(folder, f"{forecast}_{num_bits}.pkl")

with open(file, "wb") as fh:
    pickle.dump(score_d, fh)

In [14]:
# # load results
# with open(file, "rb") as fh:
#     scores_d2 = pickle.load(fh)

# scores_d2

In [15]:
score_d

{'mse': [1.3539067110705146e-06, 2.8886798108408104e-07],
 'nrmse': [0.0006042555853635369, 6.396948555827167e-05],
 'rmse': [0.0011571089524984, 0.00012249727800473864],
 'rsquare': [0.9999941712799518, 1.2436090159354816e-06]}